# Retraitement des données horaires de la qualité de l'air (format xml)

#### Nous disposons de plusieurs datasets :

- une collection de fichiers xml avec l'historique horaire de la qualité de l'air pour plusieurs région de France (1 fichier = 1 journée)
- une base de donnée B
- une base de donnée D

L'objectif est de parser les différents fichiers xml pour obtenir un tableau clair des données, pour la région qui nous intéresse (Paris)

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
#import ipdb
import datetime as dt
import calendar
import numpy as np
import ipdb

In [3]:
#Requête de tous les noms de fichier 
links=[]
years = [2018,2019,2020,2021]
for year in years:
    
    url_root = "https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/old/"+str(year)+"/"
    r = requests.get(url_root)
    soup_urls = BeautifulSoup(r.content,'xml')
    soup_urls = soup_urls.find_all('a')[1:]
    for url in soup_urls:
        links.append(url_root+url.text)
    
    i=0
    while i < len(links):
    
        
        if links[i][:-6]==links[i-1][:-6]:
            
            #ipdb.set_trace()
            if "t" in links[i][-5:]:
                links.pop(i)
            elif "t" in links[i-1][-5:]:
                links.pop(i-1)
            
        i+=1
print(len(links))
links[0:10]

21560


['https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/old/2018/fr-2018-b-lcsqa-ineris-20190724.xml',
 'https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/old/2018/fr-2018-d-lcsqa-ineris-20190918-1.xml',
 'https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/old/2018/fr-2018-d-v0.xml',
 'https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/old/2018/fr-2018-e2-2018-2018-09-12-04-t.xml',
 'https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/old/2018/fr-2018-e2-2018-2018-09-12-05-t.xml',
 'https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/old/2018/fr-2018-e2-2018-2018-09-17-14-t.xml',
 'https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/old/2018/fr-2018-e2-2018-2018-09-17-15-v.xml',
 'https://files.data.gouv.fr/lcsqa/concentrations-de-pollu

# Création du tableau de vérification

In [4]:
tbl_verification = pd.DataFrame({'code verification':[1,2,3],'description':['Verified','Preliminary verified','Not verified']})
tbl_verification

,code verification,description
0,1,Verified
1,2,Preliminary verified
2,3,Not verified


# Création du tableau de validité

In [5]:
tbl_validite = pd.DataFrame({'code verification':[-99,-1,1,2,3],
                                 'description': ['Not valid due to station maintenance or calibration in the observationvalidity vocabulary',
                                                 'Not valid',
                                                 'Valid',
                                                 'Valid, but below detection limit measurement value given in the observationvalidity vocabulary',
                                                 'Valid, but below detection limit and number replaced by 0.5*detection limit in the observationvalidity vocabulary']})
tbl_validite

,code verification,description
0,-99,Not valid due to station maintenance or calibr...
1,-1,Not valid
2,1,Valid
3,2,"Valid, but below detection limit measurement v..."
4,3,"Valid, but below detection limit and number re..."


# Récupération de la liste des polluants

In [6]:
#Récupération de la liste des polluants (ils ont préalablement été transposés depuis le site dans un fichier csv)
#La liste est non exhaustive et comprend seulement les polluants qui sont mesurés dans le dataset
tbl_polluants = pd.read_csv("polluants.csv",sep=";")

tbl_polluants.head(4)

,Code,Nom,Notation
0,1,Sulphur dioxide (air),SO2
1,3,Strong acidity (air),SA
2,4,Total suspended particulates (aerosol),SPM
3,5,Particulate matter < 10 µm (aerosol),PM10


# Récupération des données sur les points de prélèvement

In [7]:
"""
filename = "D_points_prelevement.xml"
#Attention, très lent
soup_spots = BeautifulSoup(open(filename),'xml')
"""

'\nfilename = "D_points_prelevement.xml"\n#Attention, très lent\nsoup_spots = BeautifulSoup(open(filename),\'xml\')\n'

Aperçu non exhaustif : 
    
```xml
  ...
  <gml:featureMember>
  ...
  </gml:featureMember>
  <gml:featureMember>
  ...
          <base:localId>SPO-FR01001_38</base:localId>
            ...
```
<code style="background:yellow;color:black">---> Code du point de prélèvement </code>
```xml
                   ...
                    <gn:GeographicalName>
                      ...
                      <gn:spelling>
                        <gn:SpellingOfName>
                          <gn:text>SCHILTIGHEIM</gn:text>
                        </gn:SpellingOfName>
                      </gn:spelling>
                    </gn:GeographicalName>
                  </ad:adminUnit>
                  <ad:locatorDesignator>5 rue de Madrid, Espace Européen de l'Entreprise</ad:locatorDesignator>
                  <ad:postCode>67300</ad:postCode>
               
```
<code style="background:yellow;color:black">---> Ville + Département </code>
```xml
   ...
  </gml:featureMember>

```

Le dataset des villes n'est pas assez complet. Dans les csv de ce dossier : https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/temps-reel/2021/, les codes pour paris sont : 

In [10]:
dict_codes_paris = {'PARIS1':'FR04055' ,'PARIS7': 'FR04060', 'PARIS12': 'FR04014','PARIS13':'FR04037' ,'PARIS18':'FR04004'}
dict_codes_paris

{'PARIS1': 'FR04055',
 'PARIS7': 'FR04060',
 'PARIS12': 'FR04014',
 'PARIS13': 'FR04037',
 'PARIS18': 'FR04004'}

# Récupération et retraitement des data xml sur la qualité de l'air
- https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/old/2019/
- https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/old/2020/
- https://files.data.gouv.fr/lcsqa/concentrations-de-polluants-atmospheriques-reglementes/old/2021/

#### Notes sur les différentes balises
-----------------------------------------------
```xml
<gml:TimePeriod gml:id="REP_TP_FR_2018-12-31-23_E2">
<gml:beginPosition>2018-09-19T00:00:00Z</gml:beginPosition>
<gml:endPosition>2018-10-10T00:00:00Z</gml:endPosition>
</gml:TimePeriod>
```
Période sur laquelle s'étend la donnée dans tout le fichier.

=================================================================================================

```xml
<aqd:content xlink:href="FR.LCSQA-INERIS.AQ/FR_OBS_et5tb64ner_FR070A_FR24040_7_1"/>
<aqd:content xlink:href="FR.LCSQA-INERIS.AQ/FR_OBS_86q294dv4i_FR070A_FR24039_5_2"/>
...
```
Points de prélèvement et polluants pour lesquelles les données sont fournies.

FR070A_FR24040_7 : [code reseau mesure]_ [code station]_ [n° polluant]

Numéros des polluants : http://dd.eionet.europa.eu/vocabulary/aq/pollutant/view?page=1#vocabularyConceptResults

=================================================================================================
```xml
<om:parameter>
<om:NamedValue>
<om:name xlink:href="http://dd.eionet.europa.eu/vocabulary/aq/processparameter/SamplingPoint"/>
<om:value xlink:href="FR.LCSQA-INERIS.AQ/SPO-FR24040_7"/> 
```
<code style="background:yellow;color:black">--- FR24040 : point de prélèvement</code>
```xml
</om:NamedValue>
</om:parameter>
<om:observedProperty xlink:href="http://dd.eionet.europa.eu/vocabulary/aq/pollutant/7"/> 
``` 
<code style="background:yellow;color:black">---> 7 : numéro de polluant</code>
```xml
<om:featureOfInterest xlink:href="FR.LCSQA-INERIS.AQ/SAM-FR24040_7"/>
<om:result>
<swe:DataArray>
<swe:elementCount>
<swe:Count>
<swe:value>24</swe:value>
</swe:Count>
</swe:elementCount>
``` 
<code style="background:yellow;color:black">---> 24 : nombre de données horaires disponibles sur la période de mesure</code>
```xml

<swe:elementType name="FixedObservations">
<swe:encoding>
<swe:TextEncoding blockSeparator="@@" decimalSeparator="." tokenSeparator=","/>
</swe:encoding>
<swe:values>2018-10-09T00:00:00+00:00,2018-10-09T01:00:00+00:00,2,1,61.0@@2018-10-09T01:00:00+00:00,2018-10-09T02:00:00+00:00,2,1,67.7@@2018-10-09T02:00:00+00:00,2018-10-09T03:00:00+00:00,2,1,65.0@@2018-10-09T03:00:00+00:00,2018-10-09T04:00:00+00:00,2,1,62.8@@2018-10-09T04:00:00+00:00,2018-10-09T05:00:00+00:00,2,1,66.5@@2018-10-09T05:00:00+00:00,2018-10-09T06:00:00+00:00,2,1,66.0@@</swe:values>
 ``` 
<code style="background:yellow;color:black">---> Groupe de valeurs séparées par "@@" </code>
    
<code style="background:yellow;color:black">2018-10-09T00:00:00+00:00,2018-10-09T01:00:00+00:00,2,1,61.0 = StartTime,EndTime,Verification (1), Validity (2), Value </code>

(1) voir tbl_verification


(2) voir tbl_validite

In [11]:
#Requête sur le code xml
#r = requests.get(url_xml_test)
#soup_data = BeautifulSoup(r.content,'xml')

Aperçu du résultat (non exhaustif) : 
    
```xml
...
<gml:featureMember>
...
</gml:featureMember>
<gml:featureMember>
...
<om:name xlink:href="http://dd.eionet.europa.eu/vocabulary/aq/processparameter/SamplingPoint"/>
<om:value xlink:href="FR.LCSQA-INERIS.AQ/SPO-FR24040_7"/>
...
<swe:Count>
<swe:value>24</swe:value>
</swe:Count>
...
<swe:encoding>
<swe:TextEncoding blockSeparator="@@" decimalSeparator="." tokenSeparator=","/>
</swe:encoding>
<swe:values>2018-10-09T00:00:00+00:00,2018-10-09T01:00:00+00:00,2,1,79.7@@2018-10-09T01:00:00+00:00,2018-10-09T02:00:00+00:00,2,1,85.7@@2018-10-09T02:00:00+00:00,2018-10-09T03:00:00+00:00,2,1,77.8@@2018-10-09T03:00:00+00:00,2018-10-09T04:00:00+00:00,2,1,80.8@@2018-10-09T04:00:00+00:00,2018-10-09T05:00:00+00:00,2,1,77.8@@2018-10-09T05:00:00+00:00,2018-10-09T06:00:00+00:00,2,1,73.3@@</swe:values>
...
</gml:featureMember>
```

In [15]:
start_time = dt.datetime.now()
dates = []
h_debut = []
h_fin = []
codes_prelev = []
verifs = []
valid = []
codes_pollu = []
valeurs = []
loop=0
for url in links:
    loop+=1
    if loop%500==0:
        print(str(np.round(loop/len(links)*100,0))+ '% : working on -' + url[100:])
    
    r = requests.get(url_xml_test)
    soup_data = BeautifulSoup(r.content,'xml')
    
    all_xml_data = soup_data.find_all("gml:featureMember")
    
    #Optimisation : on vérifie en amont s'il y a le code paris dans notre soupe
    paris_in_soup = False
    
    str_soup = str(soup_data)
    for code_paris in dict_codes_paris.values():
        if code_paris in str_soup:
            paris_in_soup = True
            break
        
        for data in all_xml_data[1:]:
            
            #Récupération du code du point de prélèvement et du polluant 
            ref = str(data.find_all("om:value")[1])
            lst = ref.split('-')[-1].split("\"")[0].split("_")

            code_zone_prelev = lst[0]

            if code_zone_prelev in dict_codes_paris.values():
                code_polluant = lst[1]


                #if code_zone_prelev in dict_codes_paris.values():
                #Récupération des données
                values = data.find_all("swe:values")[0].text
                values_split = values.split('@@')[:-2]

                for elem in values_split:
                    elem = elem.split(",")

                    dates.append(elem[0][:9])
                    h_debut.append(elem[0])
                    h_fin.append(elem[1])
                    codes_prelev.append(code_zone_prelev)
                    verifs.append(elem[2])
                    valid.append(elem[3])
                    codes_pollu.append(code_polluant)
                    valeurs.append(elem[4])


tbl_dataset = pd.DataFrame({"date":dates,
                           "h début":h_debut,
                            "h fin":h_fin,
                            "codes_prelev":codes_prelev,
                            "verif":verifs,
                            "valid":valid,
                            "code_polluant":codes_pollu,
                            "valeur":valeurs,
                           })
    



print("time", (dt.datetime.now() - start_time))
tbl_dataset

2.0% : working on -2018-e2-2018-2018-10-12-22-v.xml
5.0% : working on -2018-e2-2018-2018-11-04-18-t.xml
7.0% : working on -2018-e2-2018-2018-11-25-16-v.xml
9.0% : working on -2018-e2-2018-2018-12-16-19-v.xml
12.0% : working on -2019-e2-2019-2019-01-20-09-t.xml
14.0% : working on -2019-e2-2019-2019-02-10-15-v.xml
16.0% : working on -2019-e2-2019-2019-03-06-06-t.xml
19.0% : working on -2019-e2-2019-2019-03-29-14-t.xml
21.0% : working on -2019-e2-2019-2019-04-19-22-v.xml
23.0% : working on -2019-e2-2019-2019-05-10-23-v.xml
26.0% : working on -2019-e2-2019-2019-05-31-20-v.xml
28.0% : working on -2019-e2-2019-2019-06-21-16-v.xml
30.0% : working on -2019-e2-2019-2019-07-12-12-t.xml
32.0% : working on -2019-e2-2019-2019-08-04-23-t.xml
35.0% : working on -2019-e2-2019-2019-08-25-22-t.xml
37.0% : working on -2019-e2-2019-2019-09-15-18-v.xml
39.0% : working on -2019-e2-2019-2019-10-06-18-v.xml
42.0% : working on -2019-e2-2019-2019-10-27-14-t.xml
44.0% : working on -2019-e2-2019-2019-11-17-14-t.x

,date,h début,h fin,codes_prelev,verif,valid,code_polluant,valeur


Aucune donnée ne ressort pour Paris, bien que la documentation du dataset mentionnait Paris dans sa liste de points de prélèvements. Nous ne pouvions le savoir qu'en récupérant le contenu des 21 000 fichiers xml. Bien que le résultat soit quelque peu décevant, nous avons souhaité l'inclure à notre projet car il rend compte d'un aspect de la recherche de données : on peut passer un temps considérable à extraire les données d'un dataset sans garantie de résultat.